In [25]:
# コメント文の分散表現をDoc2Vecを用いて作る。
# 副生産物としてコンテキストの分散表現を得る。

from gensim.models import Word2Vec
from tensorflow import keras
import tensorflow as tf
from keras.models import load_model
import time
import glob
import os
import os.path
import random
import sys
import numpy as np
import nltk
nltk.download('punkt')

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop

sys.path.append('..')
sys.path.append('../..')

[nltk_data] Downloading package punkt to /Users/makoto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
file = glob.glob("../../repositories_cleansing/repositories2TokenWithCommentDownOnlyRawDataWithIdent20Tokens/all.txt")
print(file)

['../../repositories_cleansing/repositories2TokenWithCommentDownOnlyRawDataWithIdent20Tokens/all.txt']


In [27]:
#トークンのサイズ
tokenSize=20

#コメントのベクトルのサイズ
commentVecSize=100

# コンテキストのベクトルサイズ
contextVecSize = 100

In [28]:
comments = []
contexts = []
f = open(file[0])
lines = f.readlines()

i=0
for line in lines:
    temp_data = line.replace("\n","")
    temp_context = " ".join(temp_data.split(' ')[0:tokenSize])
    temp_comment = " ".join(temp_data.split(' ')[tokenSize:])
    morphComment = nltk.word_tokenize(temp_comment)
    morphContext = nltk.word_tokenize(temp_context)
    contexts.append(TaggedDocument(morphContext,[i]))
    comments.append(TaggedDocument(morphComment,[i]))
    i += 1
f.close()


In [29]:
print(len(comments))
print(len(contexts))

print(contexts[0])
print()
print(comments[0])

48920
48920
TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [0])

TaggedDocument(['#', '!', '/usr/bin/env', 'ruby', '#', '--', '#', 'Copyright', '2004', 'Austin', 'Ziegler', '<', 'ruby-install', '@', 'halostatue.ca', '>', '#', 'Install', 'utility', '.', 'Based', 'on', 'the', 'original', 'installation', 'script', 'for', 'rdoc', 'by', 'the', '#', 'Pragmatic', 'Programmers', '.', '#', '#', 'This', 'program', 'is', 'free', 'software', '.', 'It', 'may', 'be', 'redistributed', 'and/or', 'modified', 'under', '#', 'the', 'terms', 'of', 'the', 'GPL', 'version', '2', '(', 'or', 'later', ')', 'or', 'the', 'Ruby', 'licence', '.', '#', '#', 'Usage', '#', '--', '--', '-', '#', 'In', 'most', 'cases', ',', 'if', 'you', 'have', 'a', 'typical', 'project', 'layout', ',', 'you', 'will', 'need', 'to', 'do', '#', 'absolutely',

In [30]:
commentModel = Doc2Vec(documents=comments, vector_size=commentVecSize,min_count=3,window=5,epochs=20,dm=0)

contextModel = Doc2Vec(documents=contexts, vector_size=contextVecSize,min_count=3,window=5,epochs=20,dm=0)

In [31]:
print(comments[0])
print(commentModel.docvecs[0])

TaggedDocument(['#', '!', '/usr/bin/env', 'ruby', '#', '--', '#', 'Copyright', '2004', 'Austin', 'Ziegler', '<', 'ruby-install', '@', 'halostatue.ca', '>', '#', 'Install', 'utility', '.', 'Based', 'on', 'the', 'original', 'installation', 'script', 'for', 'rdoc', 'by', 'the', '#', 'Pragmatic', 'Programmers', '.', '#', '#', 'This', 'program', 'is', 'free', 'software', '.', 'It', 'may', 'be', 'redistributed', 'and/or', 'modified', 'under', '#', 'the', 'terms', 'of', 'the', 'GPL', 'version', '2', '(', 'or', 'later', ')', 'or', 'the', 'Ruby', 'licence', '.', '#', '#', 'Usage', '#', '--', '--', '-', '#', 'In', 'most', 'cases', ',', 'if', 'you', 'have', 'a', 'typical', 'project', 'layout', ',', 'you', 'will', 'need', 'to', 'do', '#', 'absolutely', 'nothing', 'to', 'make', 'this', 'work', 'for', 'you', '.', 'This', 'layout', 'is', ':', '#', '#', 'bin/', '#', 'executable', 'files', '--', '``', 'commands', "''", '#', 'lib/', '#', 'the', 'source', 'of', 'the', 'library', '#', '#', 'The', 'default

In [32]:
# 文章IDが0の文章と似た文章とその内積を得ることが出来る。
commentModel.docvecs.most_similar(0)

[(48893, 0.5651543140411377),
 (17908, 0.5623788833618164),
 (12744, 0.5596225261688232),
 (26200, 0.5574123859405518),
 (21598, 0.5556349754333496),
 (6831, 0.5486117005348206),
 (12, 0.5466195344924927),
 (25389, 0.5462836623191833),
 (39428, 0.5443388223648071),
 (19382, 0.543512225151062)]

In [33]:
print(comments[48893])
print(comments[6831])
print(comments[47295])

TaggedDocument(['#', 'Cleanup', 'old', 'assets', 'in', 'the', 'compile', 'directory', '.', 'By', 'default', 'it', 'will', '#', 'keep', 'the', 'latest', 'version', ',', '2', 'backups', 'created', 'within', 'the', 'past', 'hour', '.', '#', '#', 'Examples', '#', '#', 'To', 'force', 'only', '1', 'backup', 'to', 'be', 'kept', ',', 'set', 'count=1', 'and', 'age=0', '.', '#', '#', 'To', 'only', 'keep', 'files', 'created', 'within', 'the', 'last', '10', 'minutes', ',', 'set', 'count=0', 'and', '#', 'age=600', '.', '#'], [48893])
TaggedDocument(['#', 'As', 'per', 'the', 'BSD', 'manual', 'page', ',', 'set', 'if', 'this', 'is', 'a', 'directory', ',', 'or', 'if', '#', 'any', 'execute', 'bit', 'is', 'set', 'on', 'the', 'original', '(', 'unmodified', ')', 'mode', '.', '#', 'Ignored', 'otherwise', ';', 'it', 'is', '``', 'add', 'if', "''", ',', 'not', '``', 'add', 'or', 'clear', "''", '.'], [6831])
TaggedDocument(['#', 'Batch', 'file', 'deletions', ',', 'deleting', 'over', '130,000+', 'files', 'will',

In [34]:
# 文章IDが3の文章と似た文章とその内積を得ることが出来る。
print(comments[3])
commentModel.docvecs.most_similar(3)

TaggedDocument(['#', 'to_f', 'is', "n't", 'quite', 'exact', 'for', 'strings', 'like', '``', '1.5.1', "''", 'but', 'is', 'good', '#', 'enough', 'for', 'this', 'purpose', '.'], [3])


[(21750, 0.8109817504882812),
 (36751, 0.8074740171432495),
 (46428, 0.8016729354858398),
 (10683, 0.7960600852966309),
 (42717, 0.790931224822998),
 (10579, 0.7836476564407349),
 (19361, 0.7833571434020996),
 (36309, 0.7810547351837158),
 (36883, 0.7787360548973083),
 (9431, 0.7775939702987671)]

In [35]:
print(comments[21156])
print(comments[46428])
print(comments[8996])

TaggedDocument(['#', 'only', 'consume', 'for', '``', 'k4', "''"], [21156])
TaggedDocument(['#', 'Style/LambdaCall', 'checks', 'for', 'this', 'syntax'], [46428])
TaggedDocument(['#', 'OpenBSD', 'uses', 'a', 'format', 'like', '``', 'january', '1', '1970', "''"], [8996])


In [36]:
commentModel.wv.most_similar(positive=['if', 'while'], negative=['for'])

[('1.1', 0.3753271996974945),
 ('authie', 0.36641567945480347),
 ('thereof', 0.35550248622894287),
 ('composing', 0.3448277711868286),
 ("'Stuff", 0.3283698558807373),
 ('Encryption', 0.32470762729644775),
 ('rubocop', 0.31718140840530396),
 ('conform', 0.31333932280540466),
 ('dark', 0.3114791512489319),
 ('MachOStructure', 0.30941542983055115)]

In [37]:
print(contexts[0])
print(contextModel.docvecs[0])

TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [0])
[ 2.97713995e-01 -6.85936734e-02  2.68514931e-01 -1.28271878e-01
  2.30795696e-01 -2.79075149e-02  1.64558455e-01  8.59486014e-02
  2.33235240e-01  3.22632380e-02  1.17004693e-01 -9.28113684e-02
  3.76100503e-02  1.07012510e-01  2.02631742e-01 -1.70020863e-01
 -3.16465795e-02 -8.09936784e-03  7.60693029e-02 -1.47292733e-01
 -4.50780131e-02 -4.98311073e-02 -2.06342172e-02  1.12484157e-01
  4.33944426e-02  2.44559035e-01 -3.16905305e-02 -2.66788080e-02
  4.45607118e-02 -8.48076120e-03 -8.47092643e-02  1.65301353e-01
  5.19284457e-02  7.46410564e-02  1.72349855e-01  2.95050684e-02
 -1.76930472e-01  3.27142216e-02  1.00642387e-02  1.63083989e-02
 -3.35894972e-01 -1.69896986e-02 -7.26563521e-05  3.20425592e-02
 -4.54382412e-02  6.45143315e-02  1.94610842e-02

In [38]:
# 文章IDが0の文章と似た文章とその内積を得ることが出来る。
contextModel.docvecs.most_similar(0)

[(6584, 0.996588945388794),
 (10656, 0.9928668141365051),
 (46648, 0.9921393394470215),
 (43447, 0.9907289743423462),
 (10657, 0.990708589553833),
 (11211, 0.99045729637146),
 (10658, 0.9899718761444092),
 (10083, 0.9899154901504517),
 (10749, 0.9898542165756226),
 (22975, 0.989286482334137)]

In [39]:
print(contexts[6584])
print(contexts[10656])
print(contexts[45548])

TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [6584])
TaggedDocument(['require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'", 'require', "'", 'on_tstring_content', "'"], [10656])
TaggedDocument(['base_branch', '=', 'self', '.', 'pr_json', '[', '``', 'on_tstring_content', '``', ']', '[', '``', 'on_tstring_content', '``', ']', 'base_commit', '=', 'self', '.', 'pr_json'], [45548])


In [40]:
# 文章IDが3の文章と似た文章とその内積を得ることが出来る。
print(contexts[3])
contextModel.docvecs.most_similar(3)

TaggedDocument(['UNK', '=', 'facter', '.', 'version', '.', 'to_f', 'if', 'UNK', '<', 'UNK', 'puts', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'UNK'], [3])


[(9055, 0.9172937870025635),
 (9054, 0.9098228216171265),
 (11303, 0.90413898229599),
 (34035, 0.9039670825004578),
 (13436, 0.9034141898155212),
 (41762, 0.9018785357475281),
 (13412, 0.9008514285087585),
 (2416, 0.9000276327133179),
 (2874, 0.8991254568099976),
 (2708, 0.8975423574447632)]

In [41]:
print(contexts[9055])
print(contexts[9054])
print(contexts[11303])

TaggedDocument(['if', 'resource_name', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'resource_name', '}', '``', 'else', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{'], [9055])
TaggedDocument(['UNK', '<', '<', 'UNK', 'if', 'resource_name', 'UNK', '<', '<', '``', 'on_tstring_content', '#', '{', 'UNK', '}', 'on_tstring_content', '#', '{', 'resource_name', '}', '``', 'else', 'UNK', '<', '<'], [9054])
TaggedDocument(['installed_range', '=', '``', 'on_tstring_content', '#', '{', 'version', '}', 'on_tstring_content', '#', '{', 'version', '.', 'major', '}', 'on_tstring_content', '``', 'graph', '.', 'UNK', '(', "'"], [11303])


In [ ]:
contextModel.save("context2VecDoc2Vec.model")

# commentModel.save("comment2VecDoc2Vec.model")

In [43]:
contextModel.wv.most_similar(positive=['if', 'while'], negative=['for'])

[('local_ci_source', 0.35620760917663574),
 ('severityproducer', 0.3255493640899658),
 ('rubocop', 0.31718140840530396),
 ('equality_attributes', 0.31101417541503906),
 ('naming', 0.3094143867492676),
 ('add_metric', 0.3011810779571533),
 ('invited', 0.29916706681251526),
 ('suspended', 0.2958837151527405),
 ('xhtml_preamble', 0.29482078552246094),
 ('time', 0.29315832257270813)]